In [ ]:
# 必要なライブラリのインポート
import sys
import os
sys.path.append('../../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 自作モジュールのインポート
from src.data_fetcher import DataFetcher
from src.technical_indicators import TechnicalIndicators
from src.utils import DataPreprocessor, ModelUtils, BacktestUtils

# バックテスト用ライブラリ
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# 保存されたモデルとスケーラーの読み込み
model_path = "../../models/transformer_model.h5"
scaler_path = "../../models/scaler.pkl"

try:
    model = ModelUtils.load_model(model_path)
    preprocessor = DataPreprocessor()
    preprocessor.load_scaler(scaler_path)
    print("モデルとスケーラーの読み込み完了")
except Exception as e:
    print(f"モデル読み込みエラー: {e}")
    print("先にトレーニングNotebookを実行してください")


In [ ]:
# バックテスト用データの取得（新しいデータまたは別期間のデータ）
# 実際のプロジェクトでは別の期間のデータを使用
np.random.seed(123)  # 異なるシード値でテストデータを生成
dates = pd.date_range(start='2023-01-01', periods=500, freq='D')
backtest_data = pd.DataFrame({
    'Open': np.random.randn(500).cumsum() + 150,
    'High': np.random.randn(500).cumsum() + 152,
    'Low': np.random.randn(500).cumsum() + 148,
    'Close': np.random.randn(500).cumsum() + 150,
    'Volume': np.random.randn(500).cumsum() + 1500000
}, index=dates)

# 価格の整合性を確保
for i in range(len(backtest_data)):
    backtest_data.iloc[i, 1] = max(backtest_data.iloc[i, [0, 1, 2, 3]])  # High
    backtest_data.iloc[i, 2] = min(backtest_data.iloc[i, [0, 1, 2, 3]])  # Low

print(f"バックテストデータ準備完了: {len(backtest_data)}件")
backtest_data.head()


In [ ]:
# テクニカル指標の追加
df_backtest = TechnicalIndicators.add_all_indicators(backtest_data)
print(f"バックテスト用データ: {df_backtest.shape}")

# モデル予測の実行
X_backtest = preprocessor.transform(df_backtest)
X_backtest = X_backtest.reshape(X_backtest.shape[0], 1, X_backtest.shape[1])

predictions = model.predict(X_backtest)
df_backtest['prediction'] = predictions
df_backtest['signal'] = (predictions > 0.5).astype(int)

print(f"予測完了: {len(predictions)}件")
print(f"買いシグナル: {df_backtest['signal'].sum()}件")
print(f"売りシグナル: {len(df_backtest) - df_backtest['signal'].sum()}件")


In [ ]:
# バックテスト戦略の定義
class TransformerStrategy(Strategy):
    """Transformerモデルベースの取引戦略"""
    
    def init(self):
        # シグナルを取得
        self.signal = self.data.signal
    
    def next(self):
        # 現在のシグナルを取得
        current_signal = self.signal[-1]
        
        if current_signal == 1 and not self.position:
            # 買いシグナルかつポジションなしの場合、買い
            self.buy()
        elif current_signal == 0 and self.position:
            # 売りシグナルかつポジションありの場合、売り
            self.sell()

# バックテストの実行
print("バックテスト実行中...")
bt = Backtest(df_backtest[['Open', 'High', 'Low', 'Close', 'signal']], 
              TransformerStrategy, 
              cash=10000, 
              commission=.002)

result = bt.run()
print("バックテスト完了")


In [ ]:
# バックテスト結果の表示
print("=== バックテスト結果 ===")
print(result)

# パフォーマンス図表の表示
bt.plot(figsize=(15, 8))


In [ ]:
# 詳細な分析
df_analysis = BacktestUtils.calculate_returns(df_backtest, 'signal')

# Buy & Hold戦略との比較
buy_hold_metrics = BacktestUtils.calculate_metrics(df_analysis['returns'])
strategy_metrics = BacktestUtils.calculate_metrics(df_analysis['strategy_returns'])

print("\n=== Buy & Hold戦略 ===")
for key, value in buy_hold_metrics.items():
    print(f"{key}: {value:.4f}")

print("\n=== Transformer戦略 ===")
for key, value in strategy_metrics.items():
    print(f"{key}: {value:.4f}")

# 累積リターンの比較プロット
plt.figure(figsize=(12, 6))
plt.plot(df_analysis.index, df_analysis['cumulative_returns'], label='Buy & Hold', alpha=0.7)
plt.plot(df_analysis.index, df_analysis['cumulative_strategy_returns'], label='Transformer Strategy', alpha=0.7)
plt.title('Cumulative Returns Comparison')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# 取引シグナルの可視化
plt.figure(figsize=(15, 8))

# 価格チャート
plt.subplot(2, 1, 1)
plt.plot(df_backtest.index, df_backtest['Close'], label='Price', alpha=0.7)
buy_signals = df_backtest[df_backtest['signal'] == 1]
sell_signals = df_backtest[df_backtest['signal'] == 0]
plt.scatter(buy_signals.index, buy_signals['Close'], color='green', label='Buy Signal', marker='^', s=50)
plt.scatter(sell_signals.index, sell_signals['Close'], color='red', label='Sell Signal', marker='v', s=50)
plt.title('Trading Signals')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)

# 予測確率
plt.subplot(2, 1, 2)
plt.plot(df_backtest.index, df_backtest['prediction'], label='Prediction Probability', alpha=0.7)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Threshold')
plt.title('Model Prediction Probability')
plt.xlabel('Date')
plt.ylabel('Probability')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 結果の保存
result_summary = {
    'model_path': model_path,
    'backtest_period': f"{df_backtest.index[0]} to {df_backtest.index[-1]}",
    'total_return': result['Return [%]'],
    'max_drawdown': result['Max. Drawdown [%]'],
    'sharpe_ratio': result['Sharpe Ratio'],
    'total_trades': result['# Trades']
}

print("\n=== 結果サマリー ===")
for key, value in result_summary.items():
    print(f"{key}: {value}")

# 結果をCSVファイルに保存
results_path = "../../results/backtest_results.csv"
os.makedirs(os.path.dirname(results_path), exist_ok=True)
pd.DataFrame([result_summary]).to_csv(results_path, index=False)
print(f"\n結果を保存しました: {results_path}")
